<a href="https://colab.research.google.com/github/ValentinaEmili/Sign_language/blob/main/Hands_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount google drive on colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q yt_dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 79.2 MB/s eta 0:00:00


In [4]:
import pandas as pd
import cv2
import requests
import os
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import yt_dlp

In [5]:
# load the JSON data
js_file = pd.read_json("/content/drive/MyDrive/NLP/WLASL_v0.3.json")
folder = "/content/drive/MyDrive/NLP/dataset/"

training_folder = folder + "train/"
test_folder = folder + "test/"
training_video = training_folder + "video/"
test_video = test_folder + "video/"
training_images = training_folder + "images/"
test_images = test_folder + "images/"
os.makedirs(training_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)
os.makedirs(training_video, exist_ok=True)
os.makedirs(test_video, exist_ok=True)
os.makedirs(training_images, exist_ok=True)
os.makedirs(test_images, exist_ok=True)

# check the length of the directories and that they contain the same things

# all videos with fps=25

# sources:
# {'asl5200',     youtube.com
# 'aslbrick',     mp4
# 'asldeafined',  mp4
# 'asllex',       youtu.be
# 'aslpro',       swf
# 'aslsearch',    mp4
# 'aslsignbank',  mp4
# 'aslu',         youtube.com
# 'elementalasl', mov (like mp4 maybe?)
# 'handspeak',    mp4
# 'lillybauer',   youtube.com
# 'nabboud',      youtube.com
# 'northtexas',   youtube.com
# 'scott',        youtube.com
# 'signingsavvy', mp4
# 'signschool',   mp4
# 'spreadthesign',mp4
# 'startasl',     mp4
# 'valencia-asl'} youtube.com

In [ ]:
# download youtube videos

youtube_videos = ['asl5200', 'asllex', 'aslu', 'lillybauer', 'nabboud', 'northtexas', 'scott', 'valencia-asl']
not_downloaded = 0
tot_youtube_videos = 0
downloaded = []

for i, word in enumerate(tqdm(list(js_file['gloss']), desc='glosses')):
  for j, instance in enumerate(js_file['instances'][i]):

    video_id = js_file['instances'][i][j]['video_id']
    url = js_file['instances'][i][j]['url']
    source = js_file['instances'][i][j]['source']
    split = js_file['instances'][i][j]['split']
    filename = f"{word}_{video_id}.mp4"

    if source in youtube_videos:
      tot_youtube_videos += 1
      output_path = training_video if split == 'train' else test_video
      dest_path = os.path.join(output_path, filename)
      if 'youtu.be/' in url:
          continue

      if os.path.exists(dest_path):
        downloaded.append(video_id)
        continue

      ydl_opts = {
        'outtmpl': dest_path,
        'format': 'bestvideo+bestaudio/best',
        'merge_output_format': 'mp4',
        'quiet': True,
        'sleep_interval_requests': 2,
        'max_sleep_interval': 5
        }
      try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
          ydl.download([url])
          downloaded.append(video_id)
          print(video_id, 'DOWNLOADED')
      except:
        print(video_id)
        not_downloaded += 1
        continue

glosses:   0%|          | 1/2000 [00:09<5:12:49,  9.39s/it]

68208


ERROR: [youtube] HZMTQJdBZ9g: Video unavailable


68538


ERROR: [youtube] Lk2Vws8homw: Video unavailable


...

06656 DOWNLOADED


ERROR: [youtube] csFsZla1UHo: Video unavailable


07199
07200 DOWNLOADED


ERROR: [youtube] SJEkJzHd8Pg: Video unavailable
glosses:  18%|█▊        | 363/2000 [3:07:10<22:54:43, 50.39s/it]

07201
07369 DOWNLOADED


In [ ]:
print(not_downloaded, tot_youtube_videos)
print(downloaded)

In [ ]:
# download non-youtube videos (mp4 or mov)

youtube_videos = ['asl5200', 'asllex', 'aslu', 'lillybauer', 'nabboud', 'northtexas', 'scott', 'valencia-asl']

for i, word in enumerate(tqdm(list(js_file['gloss']), desc='glosses')):
  for j, instance in enumerate(js_file['instances'][i]):

    video_id = js_file['instances'][i][j]['video_id']
    url = js_file['instances'][i][j]['url']
    source = js_file['instances'][i][j]['source']

    if source not in youtube_videos:
      try:
        resp = requests.get(url, stream=True)

        if js_file['instances'][i][j]['split'] == 'train':
          dest_file = training_video + js_file['gloss'][i] + '_' + video_id +'.mp4'
          with open(dest_file, mode='wb') as f:
            f.write(resp.content)

        else:
          dest_file = test_video + js_file['gloss'][i] + '_' + video_id +'.mp4'
          with open(dest_file, mode='wb') as f:
            f.write(resp.content)
      except:
        continue

glosses: 100%|██████████| 2000/2000 [2:12:47<00:00,  3.98s/it]


In [ ]:
# Load the JSON data
js_file = pd.read_json("/content/drive/MyDrive/NLP/WLASL_v0.3.json")
folder = "/content/drive/MyDrive/NLP/dataset/"
training_folder = folder + "train/"
test_folder = folder + "test/"
training_video = training_folder + "video/"
test_video = test_folder + "video/"
training_images = training_folder + "images/"
test_images = test_folder + "images/"
os.makedirs(training_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)
os.makedirs(training_video, exist_ok=True)
os.makedirs(test_video, exist_ok=True)
os.makedirs(training_images, exist_ok=True)
os.makedirs(test_images, exist_ok=True)

for i, word in enumerate(list(js_file['gloss'])):
  for j, instance in enumerate(js_file['instances'][i]):

    video_id = js_file['instances'][i][j]['video_id']
    url = js_file['instances'][i][j]['url']
    try:
      resp = requests.get(url, stream=True)

      if js_file['instances'][i][j]['split'] == 'train':
        dest_file = training_video + js_file['gloss'][i] + '.mp4'
        with open(dest_file, mode='wb') as f:
          f.write(resp.content)

      else:
        dest_file = test_video + js_file['gloss'][i] + '.mp4'
        with open(dest_file, mode='wb') as f:
          f.write(resp.content)
    except:
      continue

In [ ]:
js_file['instances'][0]

In [ ]:
import pandas as pd
import cv2
import requests
import os
from google.colab.patches import cv2_imshow

# Load the JSON data
js_file = pd.read_json("/content/drive/MyDrive/NLP/WLASL_v0.3.json")
url = js_file['instances'][0][0]['url']
folder = "/content/drive/MyDrive/NLP/dataset/" + js_file['gloss'][0] + "/video/"
instance = js_file['instances'][0][0]['video_id']

# Create the destination folder if it doesn't exist
dest_file = folder + instance + ".mp4"
os.makedirs(os.path.dirname(dest_file), exist_ok=True)

# Download the video
resp = requests.get(url)
with open(dest_file, mode="wb") as f:
  f.write(resp.content)
# _________________________________________________________________________________________________________________________________________________
# Open the video file
cap = cv2.VideoCapture(dest_file)

# Get FPS and total frames from the video
fps = js_file['instances'][0][0]['fps']
cap.set(cv2.CAP_PROP_FPS, fps)

curr_frame = 0
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

while True:
  ret, frame = cap.read()
  if not ret:
    break
  if curr_frame % 10 == 0:
    cv2_imshow(frame)
    curr_folder = "/content/drive/MyDrive/NLP/dataset/" + js_file['gloss'][0] + "/" + instance
    os.makedirs(curr_folder, exist_ok=True)
    frame_name = curr_folder + "/" + str(curr_frame) + ".jpg"
    cv2.imwrite(frame_name, frame)
  curr_frame += 1